**Vehicle Detection Project**

The goals / steps of this project are the following:

* Perform a Histogram of Oriented Gradients (HOG) feature extraction on a labeled training set of images and train a classifier Linear SVM classifier
* Optionally, you can also apply a color transform and append binned color features, as well as histograms of color, to your HOG feature vector. 
* Note: for those first two steps don't forget to normalize your features and randomize a selection for training and testing.
* Implement a sliding-window technique and use your trained classifier to search for vehicles in images.
* Run your pipeline on a video stream (start with the test_video.mp4 and later implement on full project_video.mp4) and create a heat map of recurring detections frame by frame to reject outliers and follow detected vehicles.
* Estimate a bounding box for vehicles detected.



## Rubric Points

### Histogram of Oriented Gradients (HOG)

#### 1. Extracted HOG features.

The hog features are very important technique for get a robust signature of object against shape variations  as in real world. first define  twho fundamental characteristics of images  that can be use in hog feature extraction:

1) Image channels = 3 
2) Image size= 64X64 pixels

The udacity selection images are used for this project,this images are divided in 9988 images of  cars and 10093 images of  non-cars aprox (next figure shows the two classes samples images). 

[image1]: ./output_images/car_notcar_samples.jpg
![alt text][image1]

Each sample is a three color channel image with 64 x 64 pixels size. The code for this phase is found in file called functions.py  (lines 96 to 144 =>extract_features function). I play with color , spatial binning and HOG feature descriptors , for example for color histogram features  of a car sample  above shows like next figure:

[image2]: ./output_images/color_hist_car.jpg
![alt text][image2]
The  color histogram features  of a Not car sample  above shows like next figure:
[image3]: ./output_images/color_hist_notcar.jpg
![alt text][image3]

The above color histograms shows the frequency distributions of intensity for each channel in some color space, this is useful for visualize  distinctive regions for car and not cars samples. In the next figure I show the 3d visualization of color info distribution for diferent color spaces for car and non car samples selected previously, for RGB space :
[image4]: ./output_images/rgbcar3d.jpg
[image5]: ./output_images/hlscar3d.jpg
[image6]: ./output_images/hsvcar3d.jpg
![alt text][image4]
for HLS space :
![alt text][image5]
for HSV space :
![alt text][image6]
Thanks to the visualization in the color spaces, it is observed that there is a convenient difference between the classes in channel H of HSV or HLS in this particular samples.

The bin spatial features for car sample is shown in next figure: 
[image7]: ./output_images/carbinspatial.jpg
![alt text][image7]
For comparison pourposes the next figure  shows the bin spatial features for car (blue line) and non car (green line) sample:
[image8]: ./output_images/notcarbinspatial.jpg
![alt text][image8]


For HOG features case , I experiment  with some parameters combination, for example in the first figure the samples (car and not car) was  divided in 64 cells (8 vertically and 8 horizontally) with 8 x 8 pixels per cell, the number of orientations is selected as 9 for capture shape variations and 2 cells per block with gray channel. In the second figure another parameter combination was tested with  256 cells (16 vertically and 16 horizontally),  4 x 4 pixels per cell, the number of orientations is selected as 12 for capture shape variations and 2 cells per block: 

[image9]: ./output_images/carhog982.jpg
[image10]: ./output_images/carhog1242.jpg
[image11]: ./output_images/notcarhog982.jpg
[image12]: ./output_images/notcarhog1242.jpg
![alt text][image9]
![alt text][image10]
For not car sample:
![alt text][image11]
![alt text][image12]

The same experiments are developed for different images of the data set in order to adjust the set of parameters. 


#### 2. Final HOG parameters.

The adjust of parameters for HOG feature extraction was made from diferent ilumination and noise examples. A qualitative process from  results views shows that pixels per cell=4 and orientations=12  better defined the shape of the vehicle as shown below  (for YCrCb color space):

[image13]: ./output_images/hogparams.jpg
[image14]: ./output_images/hogparams2.jpg
noisy sampleimage:
![alt text][image13]
low noise sample image:
![alt text][image14]

For a light-colored vehicle:
[image15]: ./output_images/hogparams4.jpg
[image16]: ./output_images/hogparams3.jpg
noisy sampleimage:
![alt text][image15]
low noise sample image:
![alt text][image16]

In both cases I found that the YCrCb color space is more robust for different lighting conditions and color of the vehicle, obtaining the most amount of information of oriented gradients in its Y and Cr channels. The code for feature extraction is found in solution.ipynb in lines 37 to 79  and use extract_features function found in functions.py file at lines 96 to 144. The train parameters and SVM model is saved in lines 113 to 132



#### 3. Train a SVM classifier.

After color and HOG  feature extraction, a SVM is trained as a binary classification problem (cars-notcars) with linear kernel. first define a number of samples per class as 9988 (line 29 of solution.ipynb), the feature vector is construct from color, spatial binned and HOG features, the final fearure length is equal to  35856. This feature vectors are normalized by colums (feature dimension, lines 83 to 86 of solution.ipynb). The data train is made in lines 92 and 93 of solution.ipynb,  in this lines randomize the samples for training and split them in 80% samples for train and 20% samples for validation. In line 102  train a SVM  and  test the accuracy in line 107.

### Sliding Window Search
[image17]: ./output_images/slidewindows.jpg
[image18]: ./output_images/hogparams3.jpg

#### 1. Sliding window search.  

The sliding windows is calibrated according to the frame image in video of project , I decide  to use four scales : (285,170),(170,120),(95,95),(45,45)  all with overlap of 0.5 ,  shown it in next figure:
![alt text][image17]
In each search  the four scales are used , of course this implies greater computational complexity but the intention is to prove the effectiveness of the search (lines 45 to 55 in solution.ipynb file).

### Video Implementation


[video1]: ./video_result.mp4
[video2]: ./video_result_with_lane.mp4

#### 1. Video output.  

In order to see the results of the algorithm two videos were realized with the detection of the vehicles as well as with detection of lane (lines 88 to 92 in solution.ipynb file). The system has an acceptable behavior in the detection and tracking of nearby vehicles without the presence of false positives as shown in the following video  [link to my video result](./video_result.mp4) , the lane detection is included in the following video [link to my video result](./video_result_with_lane.mp4)


#### 2. Filter for false positives.

For each frame in video the heat map of set of boxes is obtained. First obtain the the bounding boxes resulting of apply the  search_windows function  (lines 46 to 52 in solution.ipynb) , then apply find_cars function  (line 53 in solution.ipynb) which applies hog characteristics in the image sweep  and get the result bounding box. The two sets of boxes are concatenated in an array to extract the corresponding hot maps with a threshold value of 3, this eliminate most of false positive detections in local frame (lines 55 to 59 in solution.ipynb). The bounding boxes from heat map is obtained with draw_labeled_bboxes function (line 59) and are stored in a FIFO buffer of size equal to 15 (lines 60 and 61). The final bounding boxes is obtained from buffer on which the hot map is calculated (lines 63 to 66) , the final centroids are obtained and drawn in lines 68 to 72.

The next figure show the heat maps obtained for some frames in test images folder and final bounding boxes drawn:
[image19]: ./output_images/heatmaps.jpg
![alt text][image19]

### Discussion

In present project was developed a vehicle detection pipeline using traditional techniques for detecting and tracking objects on video streams , this is  by obtaining feature vectors from sample images using feature descriptor algorithms like a color histograms or histograms of oriented gradients which implies a fine tuning of  parameter set in each phase of algorithm. The results of feature extraction are sensitive to light variations as well as perspective transforms, image quality and noise in data so that the normalization of vectors is a fundamental step for support vector machine classifier. Additionally, the kernel selection must be chosen according to the dimension of the feature vector and the number of examples so that the linear kernel is suitable for this particular case[1]. Finally a search algorithm is applied to reduce the false positives of the classification system which sweeps a selected area of the image with sliding windows of variable size for the focused extraction of characteristics.

The results obtained in this project show an adequate behavior of the system in the detection of nearby vehicles in the project video however it is difficult to detect distant vehicles on the same road and other vehicles on the opposite road which are occluded by the separation barrier, this could be improved by training the classifier with many more instances of vehicles in conditions of occlusion. Also in the video do not take into account conditions when the vehicles are very close as for example in a traffic jam , in these cases it would be necessary a wide variation in the size of the search windows which can result in a greater computational complexity.




[1]A Practical Guide to Support Vector Classification, Chih-Wei Hsu, Chih-Chung Chang, and Chih-Jen Lin, Department of Computer Science, National Taiwan University, Taipei 106, Taiwan,  May 19, 2016
